In [1]:
from source import Dataset
from source import tools
import awkward as ak
import glob
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from keras import layers

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2023-12-11 15:25:44.913070: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 15:25:45.067574: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-11 15:25:45.067611: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-12-11 15:25:46.401700: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
#selecting features 
branches_to_train = [
"cscRechitClusterX",
"cscRechitClusterY",
"cscRechitClusterZ",
"cscRechitClusterTimeWeighted",
"cscRechitClusterTimeSpreadWeightedAll",
"cscRechitClusternXY",
"cscRechitClusternZ",
"cscRechitClusterXSpread",
"cscRechitClusterYSpread",
"cscRechitClusterZSpread",
"cscRechitClusterXYSpread",
"cscRechitClusterRSpread",
"cscRechitClusterEtaPhiSpread",
"cscRechitClusterEtaSpread",
"cscRechitClusterPhiSpread",
"cscRechitClusterDeltaRSpread",
"cscRechitClusterMajorAxis",
"cscRechitClusterMinorAxis",
"cscRechitClusterSkewX",
"cscRechitClusterSkewY",
"cscRechitClusterSkewZ",
"cscRechitClusterKurtX",
"cscRechitClusterKurtY",
"cscRechitClusterKurtZ", 
"cscRechitClusterSize"
]

#energy key
truth_value = "cscRechitCluster_match_gLLP_e"

In [3]:
branches_to_train

['cscRechitClusterX',
 'cscRechitClusterY',
 'cscRechitClusterZ',
 'cscRechitClusterTimeWeighted',
 'cscRechitClusterTimeSpreadWeightedAll',
 'cscRechitClusternXY',
 'cscRechitClusternZ',
 'cscRechitClusterXSpread',
 'cscRechitClusterYSpread',
 'cscRechitClusterZSpread',
 'cscRechitClusterXYSpread',
 'cscRechitClusterRSpread',
 'cscRechitClusterEtaPhiSpread',
 'cscRechitClusterEtaSpread',
 'cscRechitClusterPhiSpread',
 'cscRechitClusterDeltaRSpread',
 'cscRechitClusterMajorAxis',
 'cscRechitClusterMinorAxis',
 'cscRechitClusterSkewX',
 'cscRechitClusterSkewY',
 'cscRechitClusterSkewZ',
 'cscRechitClusterKurtX',
 'cscRechitClusterKurtY',
 'cscRechitClusterKurtZ',
 'cscRechitClusterSize']

In [4]:
#read data
dataset_signal = Dataset.Dataset("signal")
df_signal = dataset_signal.load_df("/nfs/dust/cms/user/loewetob/ML_LLP/MDS_regression/datasets/test_bigger_dataset.h5")

Opening file  /nfs/dust/cms/user/loewetob/ML_LLP/MDS_regression/datasets/test_bigger_dataset.h5


In [5]:
#cut clusters which don't have LLP
olen = len(df_signal)
df_signal = df_signal[ df_signal["cscRechitCluster_match_gLLP"]>=1 ]
print('cut out ' + str(olen - len(df_signal)) + ' clusters, which leaves ' + str(len(df_signal)) + ' clusters containing a LLP decay for training')

cut out 10236 clusters, which leaves 344078 clusters containing a LLP decay for training


In [6]:
# Splitting the data into train, val, test
train_data, testval_data = train_test_split(df_signal, train_size=0.6)
test_data, val_data = train_test_split(testval_data, test_size=0.5)

# assigning labels (true energy values)
train_labels = np.array(train_data[truth_value])#([1.]*len(train_data))
test_labels = np.array(test_data[truth_value])#([1.]*len(test_data))
val_labels = np.array(val_data[truth_value])#([1.]*len(val_data))

# select only branches we want
train_data = train_data[branches_to_train]
test_data = test_data[branches_to_train]
val_data = val_data[branches_to_train]

# check the shape
print(" signal training features", train_data.shape,
    "\n signal training labels", train_labels.shape,
    "\n signal test features", test_data.shape,
    "\n ignal test labels", test_labels.shape,
    "\n signal val features", val_data.shape,
    "\n signal val labels", val_labels.shape)

 signal training features (206446, 25) 
 signal training labels (206446,) 
 signal test features (68816, 25) 
 ignal test labels (68816,) 
 signal val features (68816, 25) 
 signal val labels (68816,)


In [7]:
# see what training data looks like
print(train_labels)
print(train_data)

# store number of features for input shape
n_features = train_data.shape[1]

[191.20671082 195.83912659  49.03752518 ...  99.17763519 111.77108002
 109.3277359 ]
        cscRechitClusterX  cscRechitClusterY  cscRechitClusterZ  \
entry                                                             
135442          75.217720         180.255081         971.266479   
311313         -13.523779        -122.013863         602.904175   
319843         -88.876160         377.135590         703.310547   
96990          201.698593         361.727386         729.699890   
133156         214.046646        -304.128204        -961.758484   
...                   ...                ...                ...   
129493         147.595016         -57.798214         609.018005   
85410         -164.670822         188.651611        -975.440063   
239343         375.876038         385.332214         693.842712   
66571          325.425323         -87.935844         693.664917   
352243         355.241699        -253.019958        -836.984314   

        cscRechitClusterTimeWeighted  cscRe

In [8]:
#defining the model and metrics
METRICS = [
      keras.metrics.MeanSquaredError(name='Brier score'),
]


def make_model(metrics=METRICS, output_bias=None, layers = [50, 30, 20, 5, 1], n_input = n_features, learningrate = 1e-3):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    #initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
    #initializer = tf.keras.initializers.RandomUniform(minval=0., maxval=200, seed=None)
    initializer = tf.keras.initializers.Constant(value=100)
    
    model = keras.Sequential(keras.layers.Dense(layers[0], activation = 'relu', input_shape = (n_input,)))

    
    for nodes in layers[1:]:
        model.add(keras.layers.Dense(nodes, activation = 'relu'))
    

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = learningrate),
        loss = keras.losses.MeanSquaredError(name='MSE'),
        metrics = metrics
    )
    
    return model

In [9]:
# prerequesites

X_train, Y_train = sklearn.utils.shuffle(train_data, train_labels, random_state=0) # change to random_state=None for full randomness
X_test, Y_test = sklearn.utils.shuffle(test_data, test_labels, random_state=0) # change to random_state=None for full randomness
X_val, Y_val = sklearn.utils.shuffle(val_data, val_labels, random_state=0) # change to random_state=None for full randomness

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    verbose = 1,
    patience = 10,
    mode = 'min',#max
    restore_best_weights = True)

In [10]:
testing_df = pd.DataFrame({'layers': [], 'LOFeatures': [],'epochs': [],'batchsize': [], 'learnrate': [], 'train_loss': [], 'val_loss': [], 'test_loss': [], 'history': []})
print(testing_df)

Empty DataFrame
Columns: [layers, LOFeatures, epochs, batchsize, learnrate, train_loss, val_loss, test_loss, history]
Index: []


In [11]:
def plot_loss(history):
    fig, ax = plt.subplots(figsize = (6, 4))
    ax.plot(history.epoch, history.history['loss'], label = 'Training', color = 'blue')
    ax.plot(history.epoch, history.history['val_loss'], linestyle = '--', color = 'blue', label = 'Validation')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_yscale('log')
    ax.legend()
    
def predict_plot(model, test_data, test_labels, batches, title = None, axlabels = ('clusters', 'generated particle energy')):
    fig, ax = plt.subplots(figsize = (6, 4))
    
    predictions = model.predict(test_data, batch_size = batches,  verbose=0)
    histdata, bins, dummy = ax.hist(test_labels, bins = 50, histtype="step", color = 'b', label = 'truth')
    ax.hist(predictions, bins = bins, histtype="step", color = 'r', label = 'predictions')
    ax.set_yscale('log')
    ax.set_ylabel(axlabels[0])
    ax.set_xlabel(axlabels[1])
    ax.legend()
    if title:
        ax.set_title(title)

        
def modeltest(plotdatax, plotdatay, layers = [50, 30, 20, 5, 1], leftoutfeatures = [], epochs = 100, learn = 1e-3, batchsize = 1024):
    print('model layers : ', layers, 
         '\nfeatures left out : ', leftoutfeatures,
         '\nepochs : ', epochs,
         '\nbatch size : ', batchsize,
         '\nlearning rate : ', learn)
    mymodel = make_model(layers = layers, learningrate = learn)
    #mymodel.summary()
    history = mymodel.fit(
    X_train,
    Y_train,
    batch_size = batchsize,
    epochs = epochs, 
    callbacks = [early_stopping],
    validation_data = (X_val, Y_val),
    verbose = 0)
    
    
    results = mymodel.evaluate(X_train, Y_train, batch_size = batchsize, verbose=0)
    print("Loss on training: {:0.4f}".format(results[0]))
    
    results_val = mymodel.evaluate(X_val, Y_val, batch_size = batchsize, verbose=0)
    print("Loss on validation: {:0.4f}".format(results_val[0]))
    
    results_test = mymodel.evaluate(X_test, Y_test, batch_size = batchsize, verbose=0)
    print("Loss on test: {:0.4f}".format(results_test[0]))
    print("-----------------")
    
    ep = eearly_stopping.stopped_epoch
    if ep == 0:
        ep = epochs
        
    new_row = {'layers': layers, 'LOFeatures': leftoutfeatures, 'epochs': ep, 'batchsize': batchsize, 'learnrate': learn, 'train_loss': results[0], 'val_loss': results_val[0], 'test_loss': results_test[0], 'history': history}
    testing_df.loc[len(testing_df)] = new_row

    
    #plot_loss(history)
    #predict_plot(model = mymodel,  test_data = test_data, test_labels = test_labels, title = 'predictions on test data', batches = batchsize)
    #predict_plot(model = mymodel, test_data = train_data, test_labels = train_labels, title = 'predictions on training data', batches = batchsize)
    

In [12]:
#test different learning rates
lrate = 0.1
while lrate > 0.00001:
    for i in range(3):
        modeltest(plotdatax = test_data, plotdatay = test_labels, layers = [50, 30, 20, 5, 1], learn = lrate, batchsize = 1024)
    lrate = lrate / 10

model layers :  [50, 30, 20, 5, 1] 
features left out :  [] 
epochs :  100 
batch size :  1024 
learning rate :  0.1
Restoring model weights from the end of the best epoch: 34.
Epoch 44: early stopping
Loss on training: 5023.2773
Loss on validation: 4929.1631
Loss on test: 5174.1763
-----------------
model layers :  [50, 30, 20, 5, 1] 
features left out :  [] 
epochs :  100 
batch size :  1024 
learning rate :  0.1


/nfs/dust/cms/user/loewetob/anaconda3/envs/mds_regression/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


Restoring model weights from the end of the best epoch: 1.
Epoch 11: early stopping
Loss on training: 34028.9414
Loss on validation: 33805.4414
Loss on test: 34107.9336
-----------------
model layers :  [50, 30, 20, 5, 1] 
features left out :  [] 
epochs :  100 
batch size :  1024 
learning rate :  0.1
Restoring model weights from the end of the best epoch: 1.
Epoch 11: early stopping
Loss on training: 34028.9414
Loss on validation: 33805.4414
Loss on test: 34107.9336
-----------------
model layers :  [50, 30, 20, 5, 1] 
features left out :  [] 
epochs :  100 
batch size :  1024 
learning rate :  0.01
Restoring model weights from the end of the best epoch: 47.
Epoch 57: early stopping
Loss on training: 4528.0933
Loss on validation: 4505.8291
Loss on test: 4713.4717
-----------------
model layers :  [50, 30, 20, 5, 1] 
features left out :  [] 
epochs :  100 
batch size :  1024 
learning rate :  0.01
Restoring model weights from the end of the best epoch: 1.
Epoch 11: early stopping
Loss

In [13]:
testing_df

,layers,LOFeatures,epochs,batchsize,learnrate,train_loss,val_loss,test_loss,history
0,"[50, 30, 20, 5, 1]",[],43.0,1024.0,0.1000,5023.277344,4929.163086,5174.176270,<keras.callbacks.History object at 0x2af627f59...
1,"[50, 30, 20, 5, 1]",[],10.0,1024.0,0.1000,34028.941406,33805.441406,34107.933594,<keras.callbacks.History object at 0x2af73ecdf...
2,"[50, 30, 20, 5, 1]",[],10.0,1024.0,0.1000,34028.941406,33805.441406,34107.933594,<keras.callbacks.History object at 0x2af73f481...
3,"[50, 30, 20, 5, 1]",[],56.0,1024.0,0.0100,4528.093262,4505.829102,4713.471680,<keras.callbacks.History object at 0x2af7cbfcb...
4,"[50, 30, 20, 5, 1]",[],10.0,1024.0,0.0100,34028.941406,33805.441406,34107.933594,<keras.callbacks.History object at 0x2af7a2ec5...
5,"[50, 30, 20, 5, 1]",[],39.0,1024.0,0.0100,4608.373535,4559.970703,4784.137207,<keras.callbacks.History object at 0x2af7a2f2f...
6,"[50, 30, 20, 5, 1]",[],93.0,1024.0,0.0010,4506.609375,4495.426270,4706.925293,<keras.callbacks.History object at 0x2af7a2ffb...
7,"[50, 30, 20, 5, 1]",[],10.0,1024.0,0.0010,34028.941406,33805.441406,34107.933594,<keras.callbacks.History object at 0x2af7a3075...
8,"[50, 30, 20, 5, 1]",[],0.0,1024.0,0.0010,4487.553223,4512.157227,4720.228516,<keras.callbacks.History object at 0x2af7a6a34...
9,"[50, 30, 20, 5, 1]",[],0.0,1024.0,0.0001,4917.505371,4870.742676,5071.805176,<keras.callbacks.History object at 0x2af7a3137...


In [16]:
dftest = testing_df[testing_df['learnrate'] == 0.01]['train_loss']

In [24]:
print(dftest.array[0])

4528.09326171875


In [ ]:
df

In [ ]:
df